In [ ]:
!pip install git+https://github.com/johmedr/dempy.git

In [ ]:
from dempy import *

import numpy as np

import plotly.graph_objs as go
import plotly.express as px

Declare Lorenz equations with an output $v$
\begin{align}
    \dot{x} &= 18 y - 18 x\\ 
    \dot{y} &= 46.94 x - 2 x z - y\\
    \dot{z} &= 4 x y - 4 z\\
    v &= x + y + z
\end{align}

In [ ]:
# Prior expectation on the parameters
pE = np.array([18., 18., 46.92, 2., 1., 2., 4., 1., 1., 1.])

def lorenz(x, v, P):
    x0 = P[0] * x[1] - P[1] * x[0]
    x1 = P[2] * x[0] - P[3] * x[2] * x[0] - P[4] * x[1]
    x2 = P[5] * x[0] * x[1] - P[6] * x[2]
    
    return np.array([x0, x1, x2]) / 128.

def obs(x, v, P): 
    return np.array(x.T @ P[-3:])

models = [GaussianModel(
    fsymb=lorenz, 
    gsymb=obs, 
    n=3, 
    x=np.array([0.9,0.8,30])[:, None], # initial state
    pE=pE, sv=1/8., sw=1/8., m=0, 
    W=np.array([[np.exp(32)] * 3]), 
    V=np.array([[np.exp(0)]]), 
)]

hdm = HierarchicalGaussianModel(*models)

Generate some data. 

In [ ]:
nT = 1024
gen = DEMInversion(hdm, states_embedding_order=4).generate(nT)

plot_dem_generate(hdm, gen, subplots=False); 

# Store "observed" output 
y   = gen.v[:, 0, :1]

Copy model and assume we do not know the initial state. 

In [ ]:
hdm_dec = hdm.copy()

hdm_dec[0].x  = np.array([[12,13,16]])
hdm_dec[0].pC = np.diag(np.ones(pE.shape)) * np.exp(-128) # give high precision to the parameters to avoid changing them

deminv = DEMInversion(hdm_dec, states_embedding_order=5)
dec    = deminv.run(y, nE=1)

In [ ]:
plot_dem_states(hdm, dec, gen, names={'y[0,0]': 'x+y+z', 'x[0,0]': 'x', 'x[0,1]': 'y', 'x[0,2]': 'z'}, subplots=False);